# Pedro - Short Queeze Predictor

In [18]:
# Initial imports
import pandas as pd
from pathlib import Path

In [19]:
# Read the 2023_Short-Interest-Data.csv file from the Resources folder into a Pandas DataFrame
companies_df = pd.read_csv(Path("Resources/2023_Short-Interest-Data.csv"))

# Review the DataFrame
companies_df.head()

,Company Name,Symbol,Short % of Float,Price,Market Cap,Exchange,Sector,Industry,Record Date
0,Allogene Therapeutics Inc,ALLO,41.77,7.99,1.130905e+09,NAS,Healthcare,Biotechnology,2023-01-11
1,Sunnova Energy International Inc,NOVA,23.58,18.64,2.087121e+09,NY,Technology,Solar,2023-01-11
2,Petmed Express Inc,PETS,20.07,20.71,4.197917e+08,NAS,Healthcare,Pharmaceutical Retailers,2023-01-11
3,Childrens Place Inc (the,PLCE,20.59,40.53,5.945751e+08,NAS,Consumer Cyclical,Apparel Retail,2023-01-11
4,PMV Pharmaceuticals Inc. - Common Stock,PMVP,30.57,8.24,3.710472e+08,NAS,Healthcare,Biotechnology,2023-01-11


In [21]:
# Filter the DataFrame for rows with 'Record Date' equal to '2023-01-25'
filtered_df = companies_df[companies_df['Record Date'] == '2023-01-25']

# Display the filtered DataFrame
filtered_df

,Company Name,Symbol,Short % of Float,Price,Market Cap,Exchange,Sector,Industry,Record Date
56,Geo Group Inc (the) Reit,GEO,20.69,12.04,1.464425e+09,NY,Real Estate,REIT - Healthcare Facilities,2023-01-25
57,Ebix Inc,EBIX,20.99,20.19,6.246786e+08,NAS,Technology,Software - Application,2023-01-25
58,Earthstone Energy Inc Class A,ESTE,29.99,12.79,5.645506e+08,NY,Energy,Oil & Gas E&P,2023-01-25
59,Hci Group Inc,HCI,23.45,48.71,4.130608e+08,NY,Financial Services,Insurance - Property & Casualty,2023-01-25
60,Luminar Technologies Inc. - Class A Common Stock,LAZR,29.34,6.38,1.497450e+09,NAS,Technology,Software - Application,2023-01-25
61,Igm Biosciences Inc,IGMS,20.85,22.64,5.793576e+08,NAS,Healthcare,Biotechnology,2023-01-25
62,Nordstrom Inc,JWN,22.80,22.34,3.549826e+09,NY,Consumer Cyclical,Department Stores,2023-01-25
63,Kura Sushi Usa Inc - Class A,KRUS,25.24,56.53,4.194526e+08,NAS,Consumer Cyclical,Restaurants,2023-01-25
64,Kymera Therapeutics Inc. - Common Stock,KYMR,21.41,33.38,1.503101e+09,NAS,Healthcare,Biotechnology,2023-01-25
65,Desktop Metal Inc. Class A Common Stock,DM,28.20,1.76,4.501728e+08,NY,Technology,Computer Hardware,2023-01-25


In [24]:
# Using len() function
row_count = len(filtered_df)
print("Number of rows:", row_count)

Number of rows: 55


In [22]:
# Review the data types associated with the columns
filtered_df.dtypes

Company Name         object
Symbol               object
Short % of Float    float64
Price               float64
Market Cap          float64
Exchange             object
Sector               object
Industry             object
Record Date          object
dtype: object